<a href="https://colab.research.google.com/github/adarsh-saji/Deep_learning_projects/blob/main/bidirectional_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv(r'/content/drive/MyDrive/fake_news/WELFake_Dataset.csv')

In [ ]:
data.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [ ]:
data = data.rename(columns = {'Unnamed: 0': 'Id'})

In [ ]:
data = data.dropna()

In [ ]:
x = data.drop('label', axis = 1)

In [ ]:
x.shape

(71537, 3)

In [ ]:
y= data['label']

In [ ]:
y.value_counts()

1    36509
0    35028
Name: label, dtype: int64

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import one_hot
from keras.models import Sequential

In [ ]:
voc_size = 5000

In [ ]:
messages = x.copy()

In [ ]:
messages['title'][1]

'UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE “PEACEFUL” PROTESTERS…In Her Home State Of North Carolina [VIDEO]'

In [ ]:
messages.reset_index(inplace = True)

Preprocessing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  print(i)
  reviews = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  reviews = reviews.lower()
  reviews = reviews.split()

  reviews = [ps.stem(words) for words in reviews if words not in stopwords.words('english')]
  reviews = ' '.join(reviews)
  corpus.append(reviews)

In [ ]:
one_hot_rep = [one_hot(words, voc_size) for words in corpus]

In [ ]:
sent_length = 20
paded_words = pad_sequences(one_hot_rep, maxlen = sent_length, padding = 'pre')

In [ ]:
paded_words

array([[   0,    0,    0, ..., 1431, 3930, 2641],
       [   0,    0,    0, ..., 1949, 4493, 2641],
       [   0,    0,    0, ..., 3450, 4106, 2793],
       ...,
       [   0,    0,    0, ..., 2354, 1263, 3628],
       [   0,    0,    0, ..., 2278, 4558,  330],
       [   0,    0,    0, ..., 1185, 4664, 4831]], dtype=int32)

In [ ]:
embedded_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedded_features,input_length= sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# bidirectional
embedded_features = 40
model1 = Sequential()
model1.add(Embedding(voc_size,embedded_features,input_length= sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1, activation = 'sigmoid'))
model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import numpy as np
x_final = np.array(paded_words)
y_final = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_final, y_final, random_state = 42, test_size = .3)

In [ ]:
model1.fit(x_train,y_train, validation_data = (x_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
783/783 [==============================] - 34s 31ms/step - loss: 0.3094 - accuracy: 0.8644 - val_loss: 0.2546 - val_accuracy: 0.8940
Epoch 2/10
783/783 [==============================] - 10s 13ms/step - loss: 0.2149 - accuracy: 0.9145 - val_loss: 0.2457 - val_accuracy: 0.8981
Epoch 3/10
783/783 [==============================] - 7s 9ms/step - loss: 0.1816 - accuracy: 0.9289 - val_loss: 0.2468 - val_accuracy: 0.8984
Epoch 4/10
783/783 [==============================] - 9s 11ms/step - loss: 0.1559 - accuracy: 0.9393 - val_loss: 0.2744 - val_accuracy: 0.8951
Epoch 5/10
783/783 [==============================] - 8s 10ms/step - loss: 0.1315 - accuracy: 0.9498 - val_loss: 0.2775 - val_accuracy: 0.8933
Epoch 6/10
783/783 [==============================] - 10s 13ms/step - loss: 0.1117 - accuracy: 0.9564 - val_loss: 0.3096 - val_accuracy: 0.8876
Epoch 7/10
783/783 [==============================] - 7s 9ms/step - loss: 0.0911 - accuracy: 0.9657 - val_loss: 0.3648 - val_accuracy: 0.892

In [ ]:
y_pred_prob = model1.predict(x_test)
y_pred_classes = (y_pred_prob > 0.5).astype(int)


671/671 [==============================] - 3s 3ms/step


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
print('accuracy = ', accuracy_score(y_test, y_pred_classes))

accuracy =  0.8884074177616252


In [ ]:
print('confusion_matrix = ', confusion_matrix(y_test, y_pred_classes))

confusion_matrix =  [[9230 1365]
 [1030 9837]]
